<h1> 2c. Refactoring to add batching and feature-creation </h1>

In this notebook, we continue reading the same small dataset, but refactor our ML pipeline in two small, but significant, ways:
<ol>
<li> Refactor the input to read data in batches.
<li> Refactor the feature creation so that it is not one-to-one with inputs.
</ol>
The Pandas function in the previous notebook also batched, only after it had read the whole data into memory -- on a large dataset, this won't be an option.
<p><a herf=https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data>数据来源</a></p>

In [1]:
import google.cloud.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil
import tensorflow as tf
print(tf.__version__)

1.13.1


<h2> 1. Refactor the input </h2>

Read data created in Lab1a, but this time make it more general and performant.  Instead of using Pandas, we will use TensorFlow's Dataset API.

In [16]:
CSV_COLUMNS = ['key','fare_amount','pickup_datetime','pickup_longitude',
               'pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [['nokey'], [0.0], ['nokey'], [-74.0], [40.0], [-74.0], [40.7], [1.0]]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      print(f'============={value_column}=============')
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
#       label = dict(([(LABEL_COLUMN,label)]))
      return features, label

    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).skip(1).map(decode_csv)
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size) #batch 每次的数量
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn
    

def get_train():
  return read_dataset('E:/new-york-city-taxi-fare-prediction/train.csv', mode = tf.estimator.ModeKeys.TRAIN)

# def get_valid():
#   return read_dataset('E:/new-york-city-taxi-fare-prediction/taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

# def get_test():
#   return read_dataset('E:/new-york-city-taxi-fare-prediction/test.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> 2. Refactor the way features are created. </h2>

For now, pass these through (same as previous lab).  However, refactoring this way will enable us to break the one-to-one relationship between inputs and features.

In [17]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickup_longitude'),
    tf.feature_column.numeric_column('pickup_latitude'),
    tf.feature_column.numeric_column('dropoff_latitude'),
    tf.feature_column.numeric_column('dropoff_longitude'),
    tf.feature_column.numeric_column('passenger_count'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Create and train the model </h2>

Note that we train for num_steps * batch_size examples.

In [18]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)
model.train(input_fn = get_train(), steps = 100);  # TODO: change the name of input_fn as needed

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020CD5741A58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
=============Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)=============
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Do

<h3> Evaluate model </h3>

As before, evaluate on the validation data.  We'll do the third refactoring (to move the evaluation into the training loop) in the next lab.

In [26]:
CSV_COLUMNS = ['key','pickup_datetime','pickup_longitude',
               'pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [['nokey'], ['nokey'], [-74.0], [40.0], [-74.0], [40.7], [1.0]]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      print(f'============={value_column}=============')
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
#       label = None
#       label = features.pop(LABEL_COLUMN)
#       label = dict(([(LABEL_COLUMN,label)]))
      return features

    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).skip(1).map(decode_csv)
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size) #batch 每次的数量
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn
    

# def get_train():
#   return read_dataset('E:/new-york-city-taxi-fare-prediction/train.csv', mode = tf.estimator.ModeKeys.TRAIN)

# def get_valid():
#   return read_dataset('E:/new-york-city-taxi-fare-prediction/taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
  return read_dataset('E:/new-york-city-taxi-fare-prediction/test.csv', mode = tf.estimator.ModeKeys.EVAL)

In [31]:
prediction = model.predict(input_fn = get_test())
print(prediction)

<generator object EstimatorV2.predict at 0x0000020CD5051C00>


In [33]:
for i in prediction:
    print(i)

=============Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)=============
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([10.6765375], dtype=float32)}
{'predictions': array([10.676092], dtype=float32)}
{'predictions': array([10.676492], dtype=float32)}
{'predictions': array([10.677824], dtype=float32)}
{'predictions': array([10.677575], dtype=float32)}
{'predictions': array([10.67576], dtype=float32)}
{'predictions': array([10.675845], dtype=float32)}
{'predictions': array([10.663602], dtype=float32)}
{'predictions': array([10.676901], dtype=float32)}
{'predictions': array([10.67752], dtype=float32)}
{'predictions': array([10.676411], dtype=float32)}
{'predictions': array([10.676757], dtype=float32)}
{'predictions': array([10.676848], dt

{'predictions': array([10.679398], dtype=float32)}
{'predictions': array([10.6762085], dtype=float32)}
{'predictions': array([10.676737], dtype=float32)}
{'predictions': array([10.676649], dtype=float32)}
{'predictions': array([10.679954], dtype=float32)}
{'predictions': array([10.677213], dtype=float32)}
{'predictions': array([10.676649], dtype=float32)}
{'predictions': array([10.678309], dtype=float32)}
{'predictions': array([10.676533], dtype=float32)}
{'predictions': array([10.672457], dtype=float32)}
{'predictions': array([10.677045], dtype=float32)}
{'predictions': array([10.677187], dtype=float32)}
{'predictions': array([10.675875], dtype=float32)}
{'predictions': array([10.675153], dtype=float32)}
{'predictions': array([10.673256], dtype=float32)}
{'predictions': array([10.676397], dtype=float32)}
{'predictions': array([10.67788], dtype=float32)}
{'predictions': array([10.677728], dtype=float32)}
{'predictions': array([10.676409], dtype=float32)}
{'predictions': array([10.67824

{'predictions': array([10.676185], dtype=float32)}
{'predictions': array([10.676105], dtype=float32)}
{'predictions': array([10.676517], dtype=float32)}
{'predictions': array([10.677308], dtype=float32)}
{'predictions': array([10.677409], dtype=float32)}
{'predictions': array([10.6765785], dtype=float32)}
{'predictions': array([10.677151], dtype=float32)}
{'predictions': array([10.675698], dtype=float32)}
{'predictions': array([10.676614], dtype=float32)}
{'predictions': array([10.675918], dtype=float32)}
{'predictions': array([10.678052], dtype=float32)}
{'predictions': array([10.676838], dtype=float32)}
{'predictions': array([10.67566], dtype=float32)}
{'predictions': array([10.676689], dtype=float32)}
{'predictions': array([10.677193], dtype=float32)}
{'predictions': array([10.678847], dtype=float32)}
{'predictions': array([10.683589], dtype=float32)}
{'predictions': array([10.675927], dtype=float32)}
{'predictions': array([10.672384], dtype=float32)}
{'predictions': array([10.67317

{'predictions': array([10.678872], dtype=float32)}
{'predictions': array([10.671378], dtype=float32)}
{'predictions': array([10.672633], dtype=float32)}
{'predictions': array([10.675827], dtype=float32)}
{'predictions': array([10.676433], dtype=float32)}
{'predictions': array([10.678224], dtype=float32)}
{'predictions': array([10.677621], dtype=float32)}
{'predictions': array([10.675569], dtype=float32)}
{'predictions': array([10.678612], dtype=float32)}
{'predictions': array([10.675508], dtype=float32)}
{'predictions': array([10.677157], dtype=float32)}
{'predictions': array([10.676583], dtype=float32)}
{'predictions': array([10.678762], dtype=float32)}
{'predictions': array([10.676351], dtype=float32)}
{'predictions': array([10.676934], dtype=float32)}
{'predictions': array([10.678586], dtype=float32)}
{'predictions': array([10.677014], dtype=float32)}
{'predictions': array([10.675771], dtype=float32)}
{'predictions': array([10.663058], dtype=float32)}
{'predictions': array([10.67699

{'predictions': array([10.6760845], dtype=float32)}
{'predictions': array([10.6764], dtype=float32)}
{'predictions': array([10.677166], dtype=float32)}
{'predictions': array([10.674497], dtype=float32)}
{'predictions': array([10.677982], dtype=float32)}
{'predictions': array([10.676558], dtype=float32)}
{'predictions': array([10.67941], dtype=float32)}
{'predictions': array([10.674202], dtype=float32)}
{'predictions': array([10.677344], dtype=float32)}
{'predictions': array([10.662711], dtype=float32)}
{'predictions': array([10.683595], dtype=float32)}
{'predictions': array([10.676204], dtype=float32)}
{'predictions': array([10.677641], dtype=float32)}
{'predictions': array([10.677249], dtype=float32)}
{'predictions': array([10.677475], dtype=float32)}
{'predictions': array([10.677133], dtype=float32)}
{'predictions': array([10.678901], dtype=float32)}
{'predictions': array([10.676037], dtype=float32)}
{'predictions': array([10.67674], dtype=float32)}
{'predictions': array([10.676599],

...

{'predictions': array([11.027968], dtype=float32)}
{'predictions': array([11.028978], dtype=float32)}
{'predictions': array([11.02814], dtype=float32)}
{'predictions': array([11.02769], dtype=float32)}
{'predictions': array([11.029537], dtype=float32)}
{'predictions': array([11.027031], dtype=float32)}
{'predictions': array([11.027011], dtype=float32)}
{'predictions': array([11.026755], dtype=float32)}
{'predictions': array([11.027429], dtype=float32)}
{'predictions': array([11.028123], dtype=float32)}
{'predictions': array([11.028174], dtype=float32)}
{'predictions': array([11.022616], dtype=float32)}
{'predictions': array([11.027471], dtype=float32)}
{'predictions': array([11.028482], dtype=float32)}
{'predictions': array([11.027085], dtype=float32)}
{'predictions': array([11.028873], dtype=float32)}
{'predictions': array([11.028602], dtype=float32)}
{'predictions': array([11.027588], dtype=float32)}
{'predictions': array([11.024827], dtype=float32)}
{'predictions': array([11.02951],

{'predictions': array([11.028094], dtype=float32)}
{'predictions': array([11.028342], dtype=float32)}
{'predictions': array([11.027942], dtype=float32)}
{'predictions': array([11.027509], dtype=float32)}
{'predictions': array([11.026903], dtype=float32)}
{'predictions': array([11.027291], dtype=float32)}
{'predictions': array([11.027387], dtype=float32)}
{'predictions': array([11.028136], dtype=float32)}
{'predictions': array([11.193309], dtype=float32)}
{'predictions': array([11.203517], dtype=float32)}
{'predictions': array([11.204307], dtype=float32)}
{'predictions': array([11.206031], dtype=float32)}
{'predictions': array([11.204289], dtype=float32)}
{'predictions': array([11.2034855], dtype=float32)}
{'predictions': array([11.203623], dtype=float32)}
{'predictions': array([11.206127], dtype=float32)}
{'predictions': array([11.203692], dtype=float32)}
{'predictions': array([11.203544], dtype=float32)}
{'predictions': array([11.200502], dtype=float32)}
{'predictions': array([11.2050

{'predictions': array([11.204093], dtype=float32)}
{'predictions': array([11.203918], dtype=float32)}
{'predictions': array([11.202687], dtype=float32)}
{'predictions': array([11.205387], dtype=float32)}
{'predictions': array([11.204931], dtype=float32)}
{'predictions': array([11.203605], dtype=float32)}
{'predictions': array([11.20443], dtype=float32)}
{'predictions': array([11.205473], dtype=float32)}
{'predictions': array([11.203051], dtype=float32)}
{'predictions': array([11.203224], dtype=float32)}
{'predictions': array([11.204643], dtype=float32)}
{'predictions': array([11.205288], dtype=float32)}
{'predictions': array([11.201923], dtype=float32)}
{'predictions': array([11.198286], dtype=float32)}
{'predictions': array([11.20428], dtype=float32)}
{'predictions': array([11.201834], dtype=float32)}
{'predictions': array([11.205689], dtype=float32)}
{'predictions': array([11.204719], dtype=float32)}
{'predictions': array([11.203128], dtype=float32)}
{'predictions': array([11.193951]

{'predictions': array([11.3785515], dtype=float32)}
{'predictions': array([11.3785], dtype=float32)}
{'predictions': array([11.379387], dtype=float32)}
{'predictions': array([11.380522], dtype=float32)}
{'predictions': array([11.378994], dtype=float32)}
{'predictions': array([11.381451], dtype=float32)}
{'predictions': array([11.378936], dtype=float32)}
{'predictions': array([11.380776], dtype=float32)}
{'predictions': array([11.378233], dtype=float32)}
{'predictions': array([11.381238], dtype=float32)}
{'predictions': array([11.379079], dtype=float32)}
{'predictions': array([11.379209], dtype=float32)}
{'predictions': array([11.379191], dtype=float32)}
{'predictions': array([11.379212], dtype=float32)}
{'predictions': array([11.380214], dtype=float32)}
{'predictions': array([11.3808], dtype=float32)}
{'predictions': array([11.380626], dtype=float32)}
{'predictions': array([11.380292], dtype=float32)}
{'predictions': array([11.378558], dtype=float32)}
{'predictions': array([11.365], dt

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

